In [ ]:
!pip install -q transformers datasets gradio ftfy torch torchvision tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Define your model save directory
save_path = "/content/drive/My Drive/CLIP_Project/Model_Files"

Mounted at /content/drive


In [4]:
import os
import pandas as pd

# Paths
CSV_PATH = '/content/drive/My Drive/CLIP_Project/Data/results.csv'
IMG_DIR = '/content/drive/My Drive/CLIP_Project/Data/Fine_Tuning_Images'

# Step 1: Read CSV correctly
df = pd.read_csv(CSV_PATH, sep='|', engine='python', skip_blank_lines=True)
df.columns = [col.strip() for col in df.columns]  # Clean column names

# Step 2: Clean up data
df['image_name'] = df['image_name'].astype(str).str.strip()
df['comment'] = df['comment'].astype(str).str.strip()
df['comment_number'] = pd.to_numeric(df['comment_number'], errors='coerce')

# Step 3: Filter for first caption per image
df = df[df['comment_number'] == 0]

# Step 4: Create full file paths and filter only existing images
df['filepath'] = df['image_name'].apply(lambda x: os.path.join(IMG_DIR, x))
df = df[df['filepath'].apply(os.path.exists)].reset_index(drop=True)

# Step 5 (Optional): Limit to 5000 samples max
df = df.sample(n=min(5000, len(df)), random_state=42).reset_index(drop=True)

# Final check
print(f"Total valid image-caption pairs: {len(df)}")
print("Sample Data:\n", df.head())

<ipython-input-4-73c8d02e31e2>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['filepath'] = df['image_name'].apply(lambda x: os.path.join(IMG_DIR, x))


Total valid image-caption pairs: 5000
Sample Data:
       image_name  comment_number  \
0  223299142.jpg             0.0   
1  309687244.jpg             0.0   
2  316266025.jpg             0.0   
3  185075930.jpg             0.0   
4  153963868.jpg             0.0   

                                             comment  \
0  The grass has ice on it and there is a mountai...   
1  A man dressed in gray climbing a large brown r...   
2  A man wearing a tan sweater is sitting on the ...   
3  A blond boy smiles with his eyes closed , buri...   
4  An operator in a gold hard hat and orange vest...   

                                            filepath  
0  /content/drive/My Drive/CLIP_Project/Data/Fine...  
1  /content/drive/My Drive/CLIP_Project/Data/Fine...  
2  /content/drive/My Drive/CLIP_Project/Data/Fine...  
3  /content/drive/My Drive/CLIP_Project/Data/Fine...  
4  /content/drive/My Drive/CLIP_Project/Data/Fine...  


In [5]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
import torch

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [12]:
# Dataset Class
class FlickrDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx]['filepath']
        caption = self.data.iloc[idx]['comment']
        image = Image.open(image_path).convert("RGB")
        return {'image': image, 'text': caption}  # raw text and PIL image

# Collate Function
def collate_fn(batch):
    texts = [item['text'] for item in batch]
    images = [item['image'] for item in batch]

    # Use processor to pad text & process images
    inputs = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True
    )
    return inputs

# DataLoader
dataset = FlickrDataset(df)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

In [15]:
# Training
import torch.nn.functional as F

# Contrastive loss function
loss_fn = CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-6)
model.train()

for epoch in range(1):  # 1 epoch for Colab
    for batch in dataloader:
        for k in batch:
            batch[k] = batch[k].to(device)

        # Forward pass
        outputs = model(**batch)
        image_embeds = outputs.image_embeds  # [batch_size, hidden_dim]
        text_embeds = outputs.text_embeds    # [batch_size, hidden_dim]

        # Normalize embeddings
        image_embeds = F.normalize(image_embeds, p=2, dim=1)
        text_embeds = F.normalize(text_embeds, p=2, dim=1)

        # Compute cosine similarity
        logits_per_image = image_embeds @ text_embeds.t()  # [batch, batch]
        logits_per_text = text_embeds @ image_embeds.t()   # [batch, batch]

        # Labels: diagonal matches
        labels = torch.arange(image_embeds.size(0), device=device)

        # Compute loss in both directions
        loss_i = loss_fn(logits_per_image, labels)
        loss_t = loss_fn(logits_per_text, labels)
        loss = (loss_i + loss_t) / 2

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1} completed. Loss: {loss.item():.4f}")

Epoch 1 completed. Loss: 1.5125


In [16]:
model.save_pretrained(save_path)
processor.save_pretrained(save_path)

print(">>>Model and Processor saved successfully<<<")

>>>Model and Processor saved successfully<<<
